In [ ]:
# Severity_modification of unreasonable(DEWP SLP STP) data in Salm

In [1]:
import pandas as pd
import numpy as np

In [3]:
salm = pd.read_csv('/Users/evelyn/Desktop/Salm.csv')

In [4]:
salm

,Unnamed: 0,CDCID,GenusName,InitialExposure,LastExposure,TotalCases,Exposurestate,ExposureState,ExposureCounty1,ExposureCounty2,...,WDSP,MXSPD,PRCP,SNDP,Fog,Rain or Drizzle,Snow or Ice Pellets,Hail,Thunder,Tornado or Funnel Cloud
0,0,55,Salmonella,2009/1/10,2009/1/23,7,IL,Illinois,Lake,NaN,...,9.271429,14.796429,0.025000,0.0000,0.035714,0.071429,0.250000,0.0,0.000000,0.0
1,1,83,Salmonella,2009/1/11,2009/2/8,5,MA,Massachusetts,Bristol,Essex,...,5.931034,12.383744,0.061552,0.0000,0.146552,0.090517,0.237069,0.0,0.000000,0.0
2,2,176,Salmonella,2009/2/28,2009/3/7,6,IL,Illinois,Cook,NaN,...,11.187500,20.096875,0.003854,0.0375,0.031250,0.093750,0.343750,0.0,0.093750,0.0
3,3,180,Salmonella,2009/3/12,2009/3/26,5,IL,Illinois,Cook,NaN,...,7.836667,15.266667,0.005778,0.0000,0.000000,0.183333,0.033333,0.0,0.016667,0.0
4,4,334,Salmonella,2009/3/1,2009/3/8,5,AR,Arkansas,Benton,NaN,...,11.203125,18.300000,0.003750,0.0000,0.000000,0.000000,0.062500,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,936,302000,Salmonella,2021/12/16,2022/1/8,5,AZ,Arizona,Maricopa,NaN,...,3.645556,8.598102,0.049938,0.0000,0.025926,0.187500,0.000000,0.0,0.008333,0.0
937,937,302153,Salmonella,2021/2/26,2021/4/8,129,ME,Maine,Androscoggin,NaN,...,7.180952,14.859524,0.060476,0.0000,0.190476,0.309524,0.190476,0.0,0.023810,0.0
938,938,302498,Salmonella,2021/3/21,2021/3/21,11,LA,Louisiana,Franklin,NaN,...,2.240000,7.010000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
939,939,302671,Salmonella,2021/8/15,2021/8/16,8,CA,California,San Diego,NaN,...,4.323718,10.100000,0.001250,0.0000,0.000000,0.041667,0.000000,0.0,0.000000,0.0


In [5]:
#download salm_null data
salm_station = pd.read_csv('/Users/evelyn/Desktop/weather information/salm_station.csv')

salm_station = salm_station.dropna(subset = ['USAF'])
salm_station = salm_station.reset_index(drop = True)

salm_station["WBAN"] = pd.to_numeric(salm_station["WBAN"], errors='coerce').fillna('0').astype('int32')
for a in range(len(salm_station)):
    wban_unmodified = salm_station.loc[a, "WBAN"]
    if 0 < wban_unmodified < 10000: 
        salm_station.loc[a, "WBAN"] = str(wban_unmodified).zfill(5)


In [6]:

salm_null = pd.read_csv('/Users/evelyn/Desktop/weather information/salm_interpolated stations.csv')

for z in range(len(salm_null)):
    salm_null.loc[z, "USAF0"] = str(salm_null.loc[z, "USAF0"])
    salm_null.loc[z, "WBAN0"] = str(int(salm_null.loc[z, "WBAN0"]))

    if not(pd.isna(salm_null.loc[z, "USAF1"])):
        salm_null.loc[z, "USAF1"] = str(salm_null.loc[z, "USAF1"])
        salm_null.loc[z, "WBAN1"] = str(int(salm_null.loc[z, "WBAN1"]))
        
    if not(pd.isna(salm_null.loc[z, "USAF2"])):
        salm_null.loc[z, "USAF2"] = str(salm_null.loc[z, "USAF2"])
        salm_null.loc[z, "WBAN2"] = str(int(salm_null.loc[z, "WBAN2"]))

In [7]:
#three interpolation
import numpy as np
path = '/Users/evelyn/Desktop/project/salm_null/'
path0 = '/Users/evelyn/Desktop/weather information/salm_interpolated/'

store_path = '/Users/evelyn/Desktop/SALM/'
variable = ["TEMP", "DEWP", "SLP", "STP", "WDSP", "MXSPD", "MAX", "MIN","PRCP"]

for i in range(len(salm_null)):
    print('{:.4%}'.format(i/len(salm_null)))
    if(pd.isna(salm_null.loc[i, "USAF2"])):
        continue
    else:
        file = []
        startyear = str(salm_null.loc[i , "InitialExposure"])[:4]
        endyear = str(salm_null.loc[i , "LastExposure"])[:4]
        w = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"]), float(salm_null.loc[i, "W2"])]
        for yr in range(int(startyear), int(endyear)+1):
            for k in range(3):
                usaf = salm_null.loc[i, "USAF"+str(k)]
                wban = pd.to_numeric(salm_null.loc[i, "WBAN"+str(k)]).astype('int32')
                filename =  str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv" 
                file.append(filename)
            dfw = pd.read_csv(path0 + file[0])
            df0 = pd.read_csv(path + file[0])
            df1 = pd.read_csv(path + file[1])
            df2 = pd.read_csv(path + file[2])
            
            for n in range(len(dfw)):
                if(len(df0[df0.DATE.str.contains(dfw.loc[n, "DATE"])]) == 0 or len(df1[df1.DATE == dfw.loc[n, "DATE"]]) == 0 or len(df2[df2.DATE == dfw.loc[n, "DATE"]]) == 0):
                    dfw = dfw.drop(index = n)
                else:
                    index0 = df0[df0.DATE.str.contains(dfw.loc[n, "DATE"])].index.tolist()[0]
                    index1 = df1[df1.DATE == dfw.loc[n, "DATE"]].index.tolist()[0]
                    index2 = df2[df2.DATE == dfw.loc[n, "DATE"]].index.tolist()[0]
                    #对TEMP进行修改
                    TEMP = [float(df0.loc[index0, "TEMP"]), float(df1.loc[index1, "TEMP"]), float(df2.loc[index2, "TEMP"])]
                    wt = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"]), float(salm_null.loc[i, "W2"])]
                    for temp in range(3):
                        if(TEMP[temp] > 9999):
                            TEMP[temp] = 0
                            wt[temp] = 0
                    zt = sum(wt)
                    if(max(wt) > 0):
                        wt[0] = wt[0]/zt
                        wt[1] = wt[1]/zt
                        wt[2] = wt[2]/zt
                        dfw.loc[n, "TEMP"] = round(sum(np.multiply(TEMP,wt)), 2)
                    else:
                        dfw.loc[n, "TEMP"] = np.nan
                    #DEWP
                    DEWP = [float(df0.loc[index0, "DEWP"]), float(df1.loc[index1, "DEWP"]), float(df2.loc[index2, "DEWP"])]
                    wd = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"]), float(salm_null.loc[i, "W2"])]
                    for dewp in range(3):
                        if(DEWP[dewp] > 9999):
                            DEWP[dewp] = 0
                            wt[dewp] = 0
                    zd = sum(wd)
                    if(max(wd) > 0):
                        wd[0] = wd[0]/zd
                        wd[1] = wd[1]/zd
                        wd[2] = wd[2]/zd
                        dfw.loc[n, "DEWP"] = round(sum(np.multiply(DEWP,wd)), 2)
                    else:
                        dfw.loc[n, "DEWP"] = np.nan
                    #SLP
                    SLP = [float(df0.loc[index0, "SLP"]), float(df1.loc[index1, "SLP"]), float(df2.loc[index2, "SLP"])]
                    wslp = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"]), float(salm_null.loc[i, "W2"])]
                    for slp in range(3):
                        if(SLP[slp] > 9999):
                            SLP[slp] = 0
                            wslp[slp] = 0
                    zslp = sum(wslp)
                    if(max(wslp) > 0):
                        wslp[0] = wslp[0]/zslp
                        wslp[1] = wslp[1]/zslp
                        wslp[2] = wslp[2]/zslp
                        dfw.loc[n, "SLP"] = round(sum(np.multiply(SLP,wslp)), 2)
                    else:
                        dfw.loc[n, "SLP"] = np.nan
                    #STP
                    STP = [float(df0.loc[index0, "STP"]), float(df1.loc[index1, "STP"]), float(df2.loc[index2, "STP"])]
                    STP_ATTRIBUTES = [float(df0.loc[index0, "STP_ATTRIBUTES"]), float(df1.loc[index1, "STP_ATTRIBUTES"]), float(df2.loc[index2, "STP_ATTRIBUTES"])]
                    wstp = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"]), float(salm_null.loc[i, "W2"])]
                    for stp in range(3):
                        if(STP[stp] > 9999):
                            STP[stp] = 0
                            wstp[stp] = 0
                        elif(STP[stp] < 100):
                            STP[stp] = STP[stp] + 1000
                        elif(STP_ATTRIBUTES[stp] == 0):
                            STP[stp] = 0
                            wstp[stp] = 0
                    zstp = sum(wstp)
                    if(max(wstp) >0):
                        wstp[0] = wstp[0]/zstp
                        wstp[1] = wstp[1]/zstp
                        wstp[2] = wstp[2]/zstp
                        dfw.loc[n, "STP"] = round(sum(np.multiply(STP,wstp)), 2)
                    else:
                        dfw.loc[n, "STP"] = np.nan
                    #WDSP
                    WDSP = [float(df0.loc[index0, "WDSP"]), float(df1.loc[index1, "WDSP"]), float(df2.loc[index2, "WDSP"])]
                    wwdsp = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"]), float(salm_null.loc[i, "W2"])]
                    for wdsp in range(3):
                        if(WDSP[wdsp] > 999):
                            WDSP[wdsp] = 0
                            wwdsp[wdsp] = 0
                    zwsdp = sum(wwdsp)
                    if(max(wwdsp) > 0):
                        wwdsp[0] = wwdsp[0]/zwsdp
                        wwdsp[1] = wwdsp[1]/zwsdp
                        wwdsp[2] = wwdsp[2]/zwsdp
                        dfw.loc[n, "WDSP"] = round(sum(np.multiply(WDSP,wwdsp)), 2)
                    else:
                        dfw.loc[n, "WDSP"] = np.nan
                    #MXSPD
                    MXSPD = [float(df0.loc[index0, "MXSPD"]), float(df1.loc[index1, "MXSPD"]), float(df2.loc[index2, "MXSPD"])]
                    wmx = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"]), float(salm_null.loc[i, "W2"])]
                    for mxspd in range(3):
                        if(MXSPD[mxspd] > 999):
                            MXSPD[mxspd] = 0
                            wmx[mxspd] = 0
                    zmx = sum(wmx)
                    if(max(wmx) > 0):
                        wmx[0] = wmx[0]/zmx
                        wmx[1] = wmx[1]/zmx
                        wmx[2] = wmx[2]/zmx
                        dfw.loc[n, "MXSPD"] = round(sum(np.multiply(MXSPD,wmx)), 2)
                    else:
                        dfw.loc[n, "MXSPD"] = np.nan
                    #MAX
                    MAX = [float(df0.loc[index0, "MAX"]), float(df1.loc[index1, "MAX"]), float(df2.loc[index2, "MAX"])]
                    wtm = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"]), float(salm_null.loc[i, "W2"])]
                    for tmax in range(3):
                        if(MAX[tmax] > 9999):
                            MAX[tmax] = 0
                            wtm[tmax] = 0
                    ztm = sum(wtm)
                    if(max(wtm) > 0):
                        wtm[0] = wtm[0]/ztm
                        wtm[1] = wtm[1]/ztm
                        wtm[2] = wtm[2]/ztm
                        dfw.loc[n, "MAX"] = round(sum(np.multiply(MAX,wtm)), 2)
                    else:
                        dfw.loc[n, "MAX"] = np.nan
                    #MIN
                    MIN = [float(df0.loc[index0, "MIN"]), float(df1.loc[index1, "MIN"]), float(df2.loc[index2, "MIN"])]
                    wtmi = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"]), float(salm_null.loc[i, "W2"])]
                    for tmin in range(3):
                        if(MIN[tmin] > 9999):
                            MIN[tmin] = 0
                            wtmi[tmin] = 0
                    ztmi = sum(wtmi)
                    if(max(wtmi)>0):
                        wtmi[0] = wtmi[0]/ztmi
                        wtmi[1] = wtmi[1]/ztmi
                        wtmi[2] = wtmi[2]/ztmi
                        dfw.loc[n, "MIN"] = round(sum(np.multiply(MIN,wtmi)), 2)
                    else:
                        dfw.loc[n, "MIN"] = np.nan
                    #PRCP
                    PRCP = [float(df0.loc[index0, "PRCP"]), float(df1.loc[index1, "PRCP"]), float(df2.loc[index2, "PRCP"])]
                    wp = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"]), float(salm_null.loc[i, "W2"])]
                    for temp in range(3):
                        if(PRCP[temp] > 99):
                            PRCP[temp] = 0
                            wp[temp] = 0
                    zp = sum(wp)
                    if(max(wp) > 0):
                        wp[0] = wp[0]/zp
                        wp[1] = wp[1]/zp
                        wp[2] = wp[2]/zp
                        dfw.loc[n, "PRCP"] = round(sum(np.multiply(PRCP,wp)), 2)
                    else:
                        dfw.loc[n, "PRCP"] = np.nan
                
            new_usaf = salm_null.loc[i, "USAF0"]
            new_wban = pd.to_numeric(salm_null.loc[i, "WBAN0"]).astype('int32')
            new_filename = str(new_usaf)+str(new_wban) + "-" + str(yr) + "DAILY.csv"
            dfw.to_csv(store_path + new_filename)
                    

0.0000%
0.4762%
0.9524%
1.4286%
1.9048%
2.3810%
2.8571%
3.3333%
3.8095%
4.2857%
4.7619%
5.2381%
5.7143%
6.1905%
6.6667%
7.1429%
7.6190%
8.0952%
8.5714%
9.0476%
9.5238%
10.0000%
10.4762%
10.9524%
11.4286%
11.9048%
12.3810%
12.8571%
13.3333%
13.8095%
14.2857%
14.7619%
15.2381%
15.7143%
16.1905%
16.6667%
17.1429%
17.6190%
18.0952%
18.5714%
19.0476%
19.5238%
20.0000%
20.4762%
20.9524%
21.4286%
21.9048%
22.3810%
22.8571%
23.3333%
23.8095%
24.2857%
24.7619%
25.2381%
25.7143%
26.1905%
26.6667%
27.1429%
27.6190%
28.0952%
28.5714%
29.0476%
29.5238%
30.0000%
30.4762%
30.9524%
31.4286%
31.9048%
32.3810%
32.8571%
33.3333%
33.8095%
34.2857%
34.7619%
35.2381%
35.7143%
36.1905%
36.6667%
37.1429%
37.6190%
38.0952%
38.5714%
39.0476%
39.5238%
40.0000%
40.4762%
40.9524%
41.4286%
41.9048%
42.3810%
42.8571%
43.3333%
43.8095%
44.2857%
44.7619%
45.2381%
45.7143%
46.1905%
46.6667%
47.1429%
47.6190%
48.0952%
48.5714%
49.0476%
49.5238%
50.0000%
50.4762%
50.9524%
51.4286%
51.9048%
52.3810%
52.8571%
53.3333%
53.8

In [8]:
#two_interpolation
import numpy as np
path = '/Users/evelyn/Desktop/project/salm_null/'

store_path = '/Users/evelyn/Desktop/SALM/'
variable = ["TEMP", "DEWP", "SLP", "STP", "WDSP", "MXSPD", "MAX", "MIN","PRCP"]

for i in range(len(salm_null)):
    print('{:.4%}'.format(i/len(salm_null)))
    if(pd.isna(salm_null.loc[i, "USAF2"])):
        file = []
        startyear = str(salm_null.loc[i , "InitialExposure"])[:4]
        endyear = str(salm_null.loc[i , "LastExposure"])[:4]
        w = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"])]
        for yr in range(int(startyear), int(endyear)+1):
            for k in range(2):
                usaf = salm_null.loc[i, "USAF"+str(k)]
                wban = pd.to_numeric(salm_null.loc[i, "WBAN"+str(k)]).astype('int32')
                filename =  str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv" 
                file.append(filename)
            df0 = pd.read_csv(path + file[0])
            df1 = pd.read_csv(path + file[1])

            for n in range(len(df0)):
                if(len(df1[df1.DATE == df0.loc[n, "DATE"]]) == 0):
                    df0 = df0.drop(index = n)
                else:
                    index1 = df1[df1.DATE == df0.loc[n, "DATE"]].index.tolist()[0]

                    #对TEMP进行修改
                    TEMP = [float(df0.loc[n, "TEMP"]), float(df1.loc[index1, "TEMP"])]
                    wt = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"])]
                    for temp in range(2):
                        if(TEMP[temp] > 9999):
                            TEMP[temp] = 0
                            wt[temp] = 0
                    zt = sum(wt)
                    if(max(wt) > 0):
                        wt[0] = wt[0]/zt
                        wt[1] = wt[1]/zt
                        df0.loc[n, "TEMP"] = round(sum(np.multiply(TEMP,wt)), 2)
                    else:
                        df0.loc[n, "TEMP"] = np.nan
                    #DEWP
                    DEWP = [float(df0.loc[n , "DEWP"]), float(df1.loc[index1, "DEWP"])]
                    wd = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"])]
                    for dewp in range(2):
                        if(DEWP[dewp] > 9999):
                            DEWP[dewp] = 0
                            wt[dewp] = 0
                    zd = sum(wd)
                    if(max(wd) > 0):
                        wd[0] = wd[0]/zd
                        wd[1] = wd[1]/zd
                        df0.loc[n, "DEWP"] = round(sum(np.multiply(DEWP,wd)), 2)
                    else:
                        df0.loc[n, "DEWP"] = np.nan
                    #SLP
                    SLP = [float(df0.loc[n, "SLP"]), float(df1.loc[index1, "SLP"])]
                    wslp = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"])]
                    for slp in range(2):
                        if(SLP[slp] > 9999):
                            SLP[slp] = 0
                            wslp[slp] = 0
                    zslp = sum(wslp)
                    if(max(wslp) > 0):
                        wslp[0] = wslp[0]/zslp
                        wslp[1] = wslp[1]/zslp
                        df0.loc[n, "SLP"] = round(sum(np.multiply(SLP,wslp)), 2)
                    else:
                        df0.loc[n, "SLP"] = np.nan
                    #STP
                    STP = [float(df0.loc[n, "STP"]), float(df1.loc[index1, "STP"])]
                    STP_ATTRIBUTES = [float(df0.loc[n, "STP_ATTRIBUTES"]), float(df1.loc[index1, "STP_ATTRIBUTES"])]
                    wstp = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"])]
                    for stp in range(2):
                        if(STP[stp] > 9999):
                            STP[stp] = 0
                            wstp[stp] = 0
                        elif(STP[stp] < 100):
                            STP[stp] = STP[stp] + 1000
                        elif(STP_ATTRIBUTES[stp] == 0):
                            STP[stp] = 0
                            wstp[stp] = 0
                    zstp = sum(wstp)
                    if(max(wstp) >0):
                        wstp[0] = wstp[0]/zstp
                        wstp[1] = wstp[1]/zstp
                        df0.loc[n, "STP"] = round(sum(np.multiply(STP,wstp)), 2)
                    else:
                        df0.loc[n, "STP"] = np.nan
                    #WDSP
                    WDSP = [float(df0.loc[n, "WDSP"]), float(df1.loc[index1, "WDSP"])]
                    wwdsp = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"])]
                    for wdsp in range(2):
                        if(WDSP[wdsp] > 999):
                            WDSP[wdsp] = 0
                            wwdsp[wdsp] = 0
                    zwdsp = sum(wwdsp)
                    if(max(wwdsp) > 0):
                        wwdsp[0] = wwdsp[0]/zwdsp
                        wwdsp[1] = wwdsp[1]/zwdsp
                        df0.loc[n, "WDSP"] = round(sum(np.multiply(WDSP,wwdsp)), 2)
                    else:
                        df0.loc[n, "WDSP"] = np.nan
                    #MXSPD
                    MXSPD = [float(df0.loc[n, "MXSPD"]), float(df1.loc[index1, "MXSPD"])]
                    wmx = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"])]
                    for mxspd in range(2):
                        if(MXSPD[mxspd] > 999):
                            MXSPD[mxspd] = 0
                            wmx[mxspd] = 0
                    zmx = sum(wmx)
                    if(max(wmx) > 0):
                        wmx[0] = wmx[0]/zmx
                        wmx[1] = wmx[1]/zmx
                        df0.loc[n, "MXSPD"] = round(sum(np.multiply(MXSPD,wmx)), 2)
                    else:
                        df0.loc[n, "MXSPD"] = np.nan
                    #MAX
                    MAX = [float(df0.loc[n, "MAX"]), float(df1.loc[index1, "MAX"])]
                    wtm = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"])]
                    for tmax in range(2):
                        if(MAX[tmax] > 9999):
                            MAX[tmax] = 0
                            wtm[tmax] = 0
                    ztm = sum(wtm)
                    if(max(wtm) > 0):
                        wtm[0] = wtm[0]/ztm
                        wtm[1] = wtm[1]/ztm
                        df0.loc[n, "MAX"] = round(sum(np.multiply(MAX,wtm)), 2)
                    else:
                        df0.loc[n, "MAX"] = np.nan
                    #MIN
                    MIN = [float(df0.loc[n, "MIN"]), float(df1.loc[index1, "MIN"])]
                    wtmi = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"])]
                    for tmin in range(2):
                        if(MIN[tmin] > 9999):
                            MIN[tmin] = 0
                            wtmi[tmin] = 0
                    ztmi = sum(wtmi)
                    if(max(wtmi)>0):
                        wtmi[0] = wtmi[0]/ztmi
                        wtmi[1] = wtmi[1]/ztmi
                        df0.loc[n, "MIN"] = round(sum(np.multiply(MIN,wtmi)), 2)
                    else:
                        df0.loc[n, "MIN"] = np.nan
                    #PRCP
                    PRCP = [float(df0.loc[n, "PRCP"]), float(df1.loc[index1, "PRCP"])]
                    wp = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"])]
                    for temp in range(2):
                        if(PRCP[temp] > 99):
                            PRCP[temp] = 0
                            wp[temp] = 0
                    zp = sum(wp)
                    if(max(wp) > 0):
                        wp[0] = wp[0]/zp
                        wp[1] = wp[1]/zp
                        df0.loc[n, "PRCP"] = round(sum(np.multiply(PRCP,wp)), 2)
                    else:
                        df0.loc[n, "PRCP"] = np.nan
                    #SNDP
                    SNDP = [float(df0.loc[n, "SNDP"]), float(df1.loc[index1, "SNDP"])]
                    wsndp = [float(salm_null.loc[i, "W0"]), float(salm_null.loc[i, "W1"])]
                    for sn in range(2):
                        if(SNDP[sn] > 999):
                            SNDP[sn] = 0
                            wsndp[sn] = 0
                    zsndp = sum(wsndp)
                    if(max(wsndp) > 0):
                        wsndp[0] = wsndp[0]/zsndp
                        wsndp[1] = wsndp[1]/zsndp
                        df0.loc[n, "SNDP"] = round(sum(np.multiply(SNDP,w)), 1)
                    else:
                        df0.loc[n, "SNDP"] = np.nan
                    
                    #FR
                    #对FRSHTT进行修改
                    fr = [df0.loc[n, "FRSHTT"], df1.loc[index1, "FRSHTT"]]
                    if(((fr[0]%10) + (fr[1]%10)) > 1):# 个位
                        df0.loc[n, "Tornado or Funnel Cloud"] = int(1)
                    else:
                        df0.loc[n, "Tornado or Funnel Cloud"] = int(0)
                    if(((fr[0]//10%10) + (fr[1]//10%10)) > 1):#十位
                        df0.loc[n, "Thunder"] = int(1)
                    else:
                        df0.loc[n, "Thunder"] = int(0)
                    if(((fr[0]//100%10) + (fr[1]//100%10)) > 1):#百位
                        df0.loc[n, "Hail"] = int(1)
                    else:
                        df0.loc[n, "Hail"] = int(0)
                    if(((fr[0]//1000%10) + (fr[1]//1000%10))  > 1):#千位
                        df0.loc[n, "Snow or Ice Pellets"] = int(1)
                    else:
                        df0.loc[n, "Snow or Ice Pellets"] = int(0)
                    if(((fr[0]//10000 % 10) + (fr[1]//10000 % 10)) > 1):
                        df0.loc[n, "Rain or Drizzle"] = int(1)
                    else:
                        df0.loc[n, "Rain or Drizzle"] = int(0)
                    if(((fr[0]//100000 % 10) + (fr[1]//100000 % 10)) > 1):
                        df0.loc[n, "Fog"] = int(1)
                    else:
                        df0.loc[n, "Fog"] = int(0)

            new_usaf = salm_null.loc[i, "USAF0"]
            new_wban = pd.to_numeric(salm_null.loc[i, "WBAN0"]).astype('int32')
            new_filename = str(new_usaf)+str(new_wban) + "-" + str(yr) + "DAILY.csv"
            df0.to_csv(store_path + new_filename)


0.0000%
0.4762%
0.9524%
1.4286%
1.9048%
2.3810%
2.8571%
3.3333%
3.8095%
4.2857%
4.7619%
5.2381%
5.7143%
6.1905%
6.6667%
7.1429%
7.6190%
8.0952%
8.5714%
9.0476%
9.5238%
10.0000%
10.4762%
10.9524%
11.4286%
11.9048%
12.3810%
12.8571%
13.3333%
13.8095%
14.2857%
14.7619%
15.2381%
15.7143%
16.1905%
16.6667%
17.1429%
17.6190%
18.0952%
18.5714%
19.0476%
19.5238%
20.0000%
20.4762%
20.9524%
21.4286%
21.9048%
22.3810%
22.8571%
23.3333%
23.8095%
24.2857%
24.7619%
25.2381%
25.7143%
26.1905%
26.6667%
27.1429%
27.6190%
28.0952%
28.5714%
29.0476%
29.5238%
30.0000%
30.4762%
30.9524%
31.4286%
31.9048%
32.3810%
32.8571%
33.3333%
33.8095%
34.2857%
34.7619%
35.2381%
35.7143%
36.1905%
36.6667%
37.1429%
37.6190%
38.0952%
38.5714%
39.0476%
39.5238%
40.0000%
40.4762%
40.9524%
41.4286%
41.9048%
42.3810%
42.8571%
43.3333%
43.8095%
44.2857%
44.7619%
45.2381%
45.7143%
46.1905%
46.6667%
47.1429%
47.6190%
48.0952%
48.5714%
49.0476%
49.5238%
50.0000%
50.4762%
50.9524%
51.4286%
51.9048%
52.3810%
52.8571%
53.3333%
53.8

In [9]:
path = '/Users/evelyn/Desktop/weather information/salm-weather/'

path1 = '/Users/evelyn/Desktop/SALM/'

import os
import os.path
import numpy as np
import datetime

for i in range(len(salm)):
    print('{:.4%}'.format(i/len(salm)))
    
    df_outbreak = salm_station[salm_station["CDCID"] == salm.loc[i,"CDCID"]]
    df_outbreak = df_outbreak.reset_index(drop = True)
    
    df_null = salm_null[salm_null["CDCID"] == salm.loc[i,"CDCID"]]
    df_null = df_null.reset_index(drop = True)
    
    TEMP = [] ; MAX = [] ; MIN = [] ; DEWP = [] ; SLP = [] ; STP = [] ; WDSP = [] ; MXSPD = [];
    PRCP = [] ; SNDP = [];
    Fog = [] ; Rain = [] ; Snow = [] ; Hail = [] ; Thunder = [] ; Tornado = [];
    if (len(df_outbreak) > 0):
        startdate = datetime.datetime.strptime(df_outbreak.loc[0, "InitialExposure"], "%Y-%m-%d")
        enddate = datetime.datetime.strptime(df_outbreak.loc[0, "LastExposure"], "%Y-%m-%d")
    elif (len(df_null) > 0):
        startdate = datetime.datetime.strptime(df_null.loc[0, "InitialExposure"], "%Y-%m-%d")
        enddate = datetime.datetime.strptime(df_null.loc[0, "LastExposure"], "%Y-%m-%d")
    else:
        continue
        
    delta = datetime.timedelta(days=1)
    d = startdate
    
    while d <= enddate:
        yr = int(str(d)[:4])
        TEMP1 = [] ; MAX1 = [] ; MIN1 = [] ; DEWP1 = [] ; SLP1 = [] ; STP1 = [] ; WDSP1 = [] ; MXSPD1 = [];
        PRCP1 = [] ; SNDP1 = [];
        Fog1 = [] ; Rain1 = [] ; Snow1 = [] ; Hail1 = [] ; Thunder1 = [] ; Tornado1 = [];
        
        if (len(df_outbreak) > 0):
            for j in range(len(df_outbreak)):
                usaf = df_outbreak.loc[j, "USAF"]
                wban = df_outbreak.loc[j, "WBAN"]
                filename = str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv"
                if not(os.path.isfile(path+filename)):
                    continue 
                elif(sum(1 for line in open(path+filename)) < 10):
                    continue 
                else:
                    df = pd.read_csv(path + filename)
                    if(df.DATE.str.contains(str(d)[:10]).any()):
                        m = df[df.DATE == str(d)[:10]].index.tolist()[0]
                        if(df.loc[m, "TEMP"] < 9999):
                            TEMP1.append(df.loc[m, "TEMP"])
                        if(df.loc[m, "MAX"] < 9999):
                            MAX1.append(df.loc[m, "MAX"])
                        if(df.loc[m, "MIN"] < 9999):
                            MIN1.append(df.loc[m, "MIN"])
                        if(df.loc[m, "DEWP"] < 9999):
                            DEWP1.append(df.loc[m, "DEWP"])
                        if(df.loc[m, "SLP"] < 9999):
                            SLP1.append(df.loc[m, "SLP"])
                        if(df.loc[m, "STP"] < 100):
                            df.loc[m, "STP"] = df.loc[m, "STP"] + 1000
                            STP1.append(df.loc[m, "STP"])
                        elif not(df.loc[m, "STP_ATTRIBUTES"] == 0):
                            STP1.append(df.loc[m, "STP"]) 
                        if(df.loc[m, "WDSP"] < 999):
                            WDSP1.append(df.loc[m, "WDSP"])
                        if(df.loc[m, "MXSPD"] < 998):
                            MXSPD1.append(df.loc[m, "MXSPD"])
                        if(df.loc[m, "PRCP"] < 99):
                            PRCP1.append(df.loc[m, "PRCP"])

                        SNDP1.append(df.loc[m, "SNDP"]) 
                        Fog1.append(df.loc[m, "Fog"])
                        Rain1.append(df.loc[m, "Rain or Drizzle"])
                        Snow1.append(df.loc[m, "Snow or Ice Pellets"])
                        Hail1.append(df.loc[m, "Hail"])
                        Thunder1.append(df.loc[m, "Thunder"])
                        Tornado1.append(df.loc[m, "Tornado or Funnel Cloud"])
                    else:
                        continue
        if (len(df_null) > 0):
            for k in range(len(df_null)):
                usaf1 = df_null.loc[k, "USAF0"]
                wban1 = df_null.loc[k, "WBAN0"]
                filename1 = str(usaf1) + str(wban1) + "-" + str(yr) + "DAILY.csv"
                dfn = pd.read_csv(path1 + filename1)
                if(dfn.DATE.str.contains(str(d)[:10]).any()):
                    a = dfn[dfn.DATE == str(d)[:10]].index.tolist()[0]
                    if not(pd.isna(dfn.loc[a, "TEMP"])):
                        TEMP1.append(dfn.loc[a, "TEMP"]) 
                    if not(pd.isna(dfn.loc[a, "MAX"])):
                        MAX1.append(dfn.loc[a, "MAX"])
                    if not(pd.isna(dfn.loc[a, "MIN"])):
                        MIN1.append(dfn.loc[a, "MIN"])
                    if not(pd.isna(dfn.loc[a, "DEWP"])):
                        DEWP1.append(dfn.loc[a, "DEWP"])
                    if not(pd.isna(dfn.loc[a, "SLP"])):
                        SLP1.append(dfn.loc[a, "SLP"])
                    if not(pd.isna(dfn.loc[a, "STP"])):
                        STP1.append(dfn.loc[a, "STP"])
                    if not(pd.isna(dfn.loc[a, "WDSP"])):
                        WDSP1.append(dfn.loc[a, "WDSP"])
                    if not(pd.isna(dfn.loc[a, "MXSPD"])):
                        MXSPD1.append(dfn.loc[a, "MXSPD"])
                    if not(pd.isna(dfn.loc[a, "PRCP"])):
                        PRCP1.append(dfn.loc[a, "PRCP"])
                        
                    SNDP1.append(dfn.loc[a, "SNDP"])
                    Fog1.append(dfn.loc[a, "Fog"])
                    Rain1.append(dfn.loc[a, "Rain or Drizzle"])
                    Snow1.append(dfn.loc[a, "Snow or Ice Pellets"])
                    Hail1.append(dfn.loc[a, "Hail"])
                    Thunder1.append(dfn.loc[a, "Thunder"])
                    Tornado1.append(dfn.loc[a, "Tornado or Funnel Cloud"])
                else:
                    continue
            
        if not(len(TEMP1) == 0):
            TEMP.append(np.mean(TEMP1))
        if not(len(MAX1) == 0):
            MAX.append(np.mean(MAX1))
        if not (len(MIN1) == 0):
            MIN.append(np.mean(MIN1))
        if not (len(DEWP1) == 0):
            DEWP.append(np.mean(DEWP1))
        if not (len(SLP1) == 0):
            SLP.append(np.mean(SLP1))
        if not (len(STP1) == 0):
            STP.append(np.mean(STP1))
        if not (len(WDSP1) == 0):
            WDSP.append(np.mean(WDSP1))
        if not (len(MXSPD1) == 0):
            MXSPD.append(np.mean(MXSPD1))
        if not (len(PRCP1) == 0):
            PRCP.append(np.mean(PRCP1))
        if not (len(SNDP1) == 0):
            SNDP.append(np.mean(SNDP1))
        if not (len(Fog1) == 0):
            Fog.append(np.mean(Fog1))
        if not (len(Rain1) == 0):
            Rain.append(np.mean(Rain1))
        if not (len(Snow1) == 0):
            Snow.append(np.mean(Snow1))
        if not (len(Hail1) == 0):
            Hail.append(np.mean(Hail1))
        if not (len(Thunder1) == 0):
            Thunder.append(np.mean(Thunder1))
        if not (len(Tornado1) == 0):
            Tornado.append(np.mean(Tornado1))
        
        d = d + delta
        
    salm.loc[i, "TEMP"] = np.mean(TEMP); 
    salm.loc[i, "MAX"] = np.mean(MAX); 
    salm.loc[i, "MIN"] = np.mean(MIN);
    salm.loc[i, "DEWP"] = np.mean(DEWP); 
    salm.loc[i, "SLP"] = np.mean(SLP); 
    salm.loc[i, "STP"] = np.mean(STP);
    salm.loc[i, "WDSP"] = np.mean(WDSP); 
    salm.loc[i, "MXSPD"] = np.mean(MXSPD); 
    salm.loc[i, "PRCP"] = np.mean(PRCP);
    salm.loc[i, "SNDP"] = np.mean(SNDP); 
    salm.loc[i, "Fog"] = np.mean(Fog); 
    salm.loc[i, "Rain or Drizzle"] = np.mean(Rain);
    salm.loc[i, "Snow or Ice Pellets"] = np.mean(Snow); 
    salm.loc[i, "Hail"] = np.mean(Hail); 
    salm.loc[i, "Thunder"] = np.mean(Thunder); 
    salm.loc[i, "Tornado or Funnel Cloud"] = np.mean(Tornado);  
    

0.0000%
0.1063%
0.2125%
0.3188%
0.4251%
0.5313%
0.6376%
0.7439%


/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0.8502%
0.9564%
1.0627%
1.1690%
1.2752%


/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1.3815%
1.4878%
1.5940%


...

96.0680%
96.1743%
96.2806%
96.3868%
96.4931%
96.5994%
96.7056%
96.8119%
96.9182%
97.0244%
97.1307%
97.2370%
97.3433%
97.4495%
97.5558%
97.6621%
97.7683%
97.8746%
97.9809%
98.0871%
98.1934%
98.2997%
98.4060%
98.5122%


/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


98.6185%
98.7248%
98.8310%


/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


98.9373%
99.0436%
99.1498%
99.2561%
99.3624%
99.4687%
99.5749%
99.6812%
99.7875%
99.8937%


In [98]:
salm.isnull().sum()

CDCID                        0
GenusName                    0
InitialExposure              0
LastExposure                 0
TotalCases                   0
Exposurestate                1
ExposureState                0
ExposureCounty1              0
ExposureCounty2            841
ExposureCounty3            867
ExposureCounty4            890
ExposureCounty5            901
ExposureCounty6            909
ExposureCounty7            919
ExposureCounty8            924
ExposureCounty9            927
ExposureCounty10           928
ExposureCounty11           931
ExposureCounty12           932
ExposureCounty13           936
ExposureCounty14           938
ExposureCounty15           938
ExposureCounty16           938
ExposureCounty17           939
ExposureCounty18           939
ExposureCounty19           940
TEMP                         0
MAX                          0
MIN                          0
DEWP                        11
SLP                        128
STP                        117
WDSP    

In [99]:
salm.to_csv('/Users/evelyn/Desktop/Salm.csv')

In [10]:
salm.describe()

,Unnamed: 0,CDCID,TotalCases,TEMP,MAX,MIN,DEWP,SLP,STP,WDSP,MXSPD,PRCP,SNDP,Fog,Rain or Drizzle,Snow or Ice Pellets,Hail,Thunder,Tornado or Funnel Cloud
count,941.000000,941.000000,941.000000,941.000000,941.000000,941.000000,930.000000,813.000000,908.000000,936.000000,936.000000,926.000000,923.000000,941.000000,941.000000,941.000000,941.000000,941.000000,941.000000
mean,470.000000,146218.775770,23.366631,62.790429,74.073344,53.081329,48.881085,1015.836696,984.238068,5.945341,12.872153,0.081203,0.038735,0.064600,0.282531,0.032484,0.000376,0.074980,0.000083
std,271.787601,134369.451493,39.053870,15.124396,15.840295,14.964454,15.322277,4.482611,40.969555,2.528293,3.739216,0.189885,0.333721,0.148762,0.297287,0.127395,0.005829,0.175728,0.001558
min,0.000000,55.000000,5.000000,9.488095,17.721212,-1.007143,1.379762,988.800000,657.562500,0.684000,4.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,235.000000,10706.000000,7.000000,53.836364,64.940896,43.184000,38.875000,1013.111364,979.547401,4.272807,10.362500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,470.000000,246466.000000,12.000000,65.625000,77.254545,55.600000,50.878271,1015.750000,993.306190,5.722030,12.604762,0.025431,0.000000,0.000000,0.235507,0.000000,0.000000,0.000000,0.000000
75%,705.000000,276713.000000,23.000000,73.870000,85.100000,64.170345,60.791667,1018.350000,1008.568690,7.304808,14.760164,0.101146,0.000000,0.066667,0.428571,0.000000,0.000000,0.071429,0.000000
max,940.000000,302902.000000,557.000000,97.981000,109.388542,84.103000,77.332143,1038.700000,1024.200000,33.000000,45.800000,3.555000,5.100000,1.000000,1.000000,1.000000,0.166667,1.000000,0.033333


In [114]:
a = []
for i in range(len(salm)):
    if(salm.loc[i, "STP"] < 700):
        a.append(salm.loc[i, "CDCID"])
print(a)

[270862]


In [12]:
salm.to_csv('/Users/evelyn/Desktop/weather information/SALM.csv')